## Very messy refactoring of data into epiweeks

In this notebook I shaped the following data into epiweeks between 2004-2023:
- ILI cases (MDSS)
- ILI ED visits (MSSS)
- average daily max windspeed
- average daily precipitation
- average daily snowfall
- average daily temperature
- AQI
- carbon monoxide
- ozone
- PM10
- PM25
- nitrogen dioxide
- 'flu' Google trends
- 'cough' Google trends
- 'cold' Google trends

output file is 'dataset by week .csv'

In [13]:
import pandas as pd
from datetime import datetime
from epiweeks import *

In [30]:
c_df = pd.read_csv('daily_aqi_data_1999_to_2023.csv')[['Date','Overall AQI Value','Main Pollutant','CO','Ozone','PM10','PM25','NO2']]

In [32]:
c_df.rename(columns={'Overall AQI Value':'aqi','Main Pollutant':'main_pollutant'},inplace=True)

In [33]:
c_df

,Date,aqi,main_pollutant,CO,Ozone,PM10,PM25,NO2
0,01/01/1999,8,PM10,5,0,8,0,-
1,01/02/1999,26,PM2.5,6,0,3,26,-
2,01/03/1999,23,PM2.5,6,0,2,23,-
3,01/04/1999,6,CO,6,0,3,0,-
4,01/05/1999,8,PM10,7,0,8,0,-
...,...,...,...,...,...,...,...,...
8967,09/09/2023,48,PM2.5,.,36,16,48,0
8968,09/10/2023,52,PM2.5,.,41,17,52,0
8969,09/11/2023,35,PM2.5,.,22,13,35,0
8970,09/12/2023,22,Ozone,.,22,8,20,0


In [36]:
c_df['Date'] = pd.to_datetime(c_df['Date']).apply(lambda x: Week.fromdate(x))

In [39]:
c_df = c_df.rename(columns={'Date':'epiweek'}).set_index('epiweek')

In [40]:
c_df

,aqi,main_pollutant,CO,Ozone,PM10,PM25,NO2
epiweek,,,,,,,
199852,8,PM10,5,0,8,0,-
199852,26,PM2.5,6,0,3,26,-
199901,23,PM2.5,6,0,2,23,-
199901,6,CO,6,0,3,0,-
199901,8,PM10,7,0,8,0,-
...,...,...,...,...,...,...,...
202336,48,PM2.5,.,36,16,48,0
202337,52,PM2.5,.,41,17,52,0
202337,35,PM2.5,.,22,13,35,0


In [48]:
import numpy as np  

In [52]:
c_df['CO'] = c_df['CO'].apply(lambda x: np.nan if x == '.' else int(x))

In [54]:
c_df['NO2'] = c_df['NO2'].apply(lambda x: np.nan if x == '-' else int(x))

In [58]:
c_df = c_df.reset_index().groupby('epiweek')[['aqi','CO','Ozone','PM10','PM25','NO2']].mean().reset_index()

In [60]:
c_df

,epiweek,aqi,CO,Ozone,PM10,PM25,NO2
0,199852,17.000000,5.500000,0.000000,5.500000,13.000000,NaN
1,199901,31.714286,8.857143,0.000000,5.428571,27.714286,NaN
2,199902,53.000000,10.428571,0.000000,10.857143,50.714286,NaN
3,199903,68.714286,17.714286,0.000000,12.857143,68.714286,NaN
4,199904,63.142857,10.571429,0.000000,15.428571,63.142857,NaN
...,...,...,...,...,...,...,...
1285,202333,51.714286,NaN,35.428571,20.142857,49.714286,0.0
1286,202334,54.571429,NaN,44.714286,20.857143,49.142857,0.0
1287,202335,42.857143,NaN,40.285714,14.428571,33.857143,0.0
1288,202336,47.000000,NaN,44.285714,16.142857,38.428571,0.0


In [69]:
c_df[['aqi','CO','Ozone','PM10','PM25','NO2']] = c_df[['aqi','CO','Ozone','PM10','PM25','NO2']].apply(lambda x: round(x))

In [71]:
epiweek_aqi_df = c_df.set_index('epiweek')

In [72]:
epiweek_aqi_df

,aqi,CO,Ozone,PM10,PM25,NO2
epiweek,,,,,,
199852,17.0,6.0,0.0,6.0,13.0,NaN
199901,32.0,9.0,0.0,5.0,28.0,NaN
199902,53.0,10.0,0.0,11.0,51.0,NaN
199903,69.0,18.0,0.0,13.0,69.0,NaN
199904,63.0,11.0,0.0,15.0,63.0,NaN
...,...,...,...,...,...,...
202333,52.0,NaN,35.0,20.0,50.0,0.0
202334,55.0,NaN,45.0,21.0,49.0,0.0
202335,43.0,NaN,40.0,14.0,34.0,0.0


In [100]:
d_df = pd.read_csv('daily_weather.csv')[['DATE','AWND','PRCP','SNOW','TAVG','TSUN']]

In [101]:
d_df.rename(columns={'DATE':'epiweek','PRCP':'precip','SNOW':'snow'},inplace=True)

In [102]:
d_df['epiweek'] = pd.to_datetime(d_df['epiweek']).apply(lambda x: Week.fromdate(x))

In [104]:
d_df = d_df.groupby('epiweek').mean()

In [89]:
d_df = d_df.apply(lambda x: round(x))

In [105]:
d_df.rename(columns={'total_sun':'total_sun_minutes'},inplace=True)

In [106]:
d_df

,AWND,precip,snow,TAVG,TSUN
epiweek,,,,,
199952,8.500000,0.000000,0.000000,36.000000,NaN
200001,12.240000,0.048571,0.323810,31.428571,NaN
200002,10.994286,0.060476,0.219048,29.428571,NaN
200003,10.288571,0.049048,0.590476,17.571429,0.0
200004,8.310000,0.017143,0.361905,14.142857,0.0
...,...,...,...,...,...
202331,6.708571,0.000429,0.000000,71.714286,NaN
202332,8.852857,0.132500,0.000000,69.571429,NaN
202333,9.267143,0.346263,0.000000,66.428571,NaN


In [107]:
d_df.max()

AWND       16.010000
precip      0.773026
snow        4.109524
TAVG       82.428571
TSUN      705.000000
dtype: float64

In [109]:
d_df.rename(columns={'snow':'av_daily_snow','av_temp':'av_daily_temp','total_sun_minutes':'av_daily_sun_minutes'},inplace=True)

In [111]:
d_df.rename(columns={'snow':'av_daily_snow','TAVG':'av_daily_temp','TSUN':'av_daily_sun_minutes','AWND':'av_daily_wind_max',},inplace=True)

In [112]:
d_df

,av_daily_wind_max,precip,av_daily_snow,av_daily_temp,av_daily_sun_minutes
epiweek,,,,,
199952,8.500000,0.000000,0.000000,36.000000,NaN
200001,12.240000,0.048571,0.323810,31.428571,NaN
200002,10.994286,0.060476,0.219048,29.428571,NaN
200003,10.288571,0.049048,0.590476,17.571429,0.0
200004,8.310000,0.017143,0.361905,14.142857,0.0
...,...,...,...,...,...
202331,6.708571,0.000429,0.000000,71.714286,NaN
202332,8.852857,0.132500,0.000000,69.571429,NaN
202333,9.267143,0.346263,0.000000,66.428571,NaN


In [121]:
epiweek_weather_aqi_df = d_df.drop(columns=['av_daily_sun_minutes']).join(epiweek_aqi_df, how='inner') 

In [124]:
epiweek_weather_aqi_df[['av_daily_wind_max','av_daily_temp']] = epiweek_weather_aqi_df[['av_daily_wind_max','av_daily_temp']].apply(lambda x: round(x))

In [127]:
epiweek_weather_aqi_df[['precip','av_daily_snow']] = epiweek_weather_aqi_df[['precip','av_daily_snow']].apply(lambda x: round(x,2))

In [128]:
epiweek_weather_aqi_df

,av_daily_wind_max,precip,av_daily_snow,av_daily_temp,aqi,CO,Ozone,PM10,PM25,NO2
epiweek,,,,,,,,,,
199952,8.0,0.00,0.00,36.0,38.0,9.0,0.0,3.0,38.0,NaN
200001,12.0,0.05,0.32,31.0,51.0,7.0,0.0,1.0,51.0,NaN
200002,11.0,0.06,0.22,29.0,54.0,7.0,0.0,1.0,54.0,NaN
200003,10.0,0.05,0.59,18.0,48.0,9.0,0.0,2.0,48.0,NaN
200004,8.0,0.02,0.36,14.0,64.0,15.0,0.0,2.0,64.0,NaN
...,...,...,...,...,...,...,...,...,...,...
202331,7.0,0.00,0.00,72.0,60.0,NaN,53.0,19.0,49.0,0.0
202332,9.0,0.13,0.00,70.0,52.0,NaN,44.0,19.0,48.0,0.0
202333,9.0,0.35,0.00,66.0,52.0,NaN,35.0,20.0,50.0,0.0


In [129]:
e_df = pd.read_csv("daily syndromic ILICC.csv")

In [131]:
e_df['epiweek'] = pd.to_datetime(e_df['Admitted']).apply(lambda x: Week.fromdate(x))

In [230]:
epiweek_weather_aqi_syn_df = pd.DataFrame(e_df.groupby('epiweek').count()['Admitted']).join(epiweek_weather_aqi_df,how='right')

In [231]:
epiweek_weather_aqi_syn_df

,Admitted,av_daily_wind_max,precip,av_daily_snow,av_daily_temp,aqi,CO,Ozone,PM10,PM25,NO2
epiweek,,,,,,,,,,,
199952,NaN,8.0,0.00,0.00,36.0,38.0,9.0,0.0,3.0,38.0,NaN
200001,NaN,12.0,0.05,0.32,31.0,51.0,7.0,0.0,1.0,51.0,NaN
200002,NaN,11.0,0.06,0.22,29.0,54.0,7.0,0.0,1.0,54.0,NaN
200003,NaN,10.0,0.05,0.59,18.0,48.0,9.0,0.0,2.0,48.0,NaN
200004,NaN,8.0,0.02,0.36,14.0,64.0,15.0,0.0,2.0,64.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
202331,36.0,7.0,0.00,0.00,72.0,60.0,NaN,53.0,19.0,49.0,0.0
202332,36.0,9.0,0.13,0.00,70.0,52.0,NaN,44.0,19.0,48.0,0.0
202333,42.0,9.0,0.35,0.00,66.0,52.0,NaN,35.0,20.0,50.0,0.0


In [234]:
h_df = pd.read_csv("ILI cases .csv")

In [235]:
h_df

,week,year,cases
0,1,2000,97
1,2,2000,1719
2,3,2000,632
3,4,2000,1245
4,5,2000,1263
...,...,...,...
1243,48,2023,0
1244,49,2023,0
1245,50,2023,0
1246,51,2023,0


In [236]:
h_df['week'] = h_df['week'].astype('str')

In [237]:
h_df['week'] = h_df['week'].apply(lambda x: '0'+x if len(x)==1 else x )

In [238]:
h_df['year'] = h_df['year'].astype('str')

In [239]:
h_df['epiweek'] = h_df['year'] + h_df['week']

In [240]:
h_df['epiweek'] = h_df['epiweek'].apply(lambda x: Week.fromstring(x))

In [245]:
df = h_df.set_index('epiweek')[['cases']].join(epiweek_weather_aqi_syn_df, how='inner')

In [246]:
df

,cases,Admitted,av_daily_wind_max,precip,av_daily_snow,av_daily_temp,aqi,CO,Ozone,PM10,PM25,NO2
epiweek,,,,,,,,,,,,
200001,97,NaN,12.0,0.05,0.32,31.0,51.0,7.0,0.0,1.0,51.0,NaN
200002,1719,NaN,11.0,0.06,0.22,29.0,54.0,7.0,0.0,1.0,54.0,NaN
200003,632,NaN,10.0,0.05,0.59,18.0,48.0,9.0,0.0,2.0,48.0,NaN
200004,1245,NaN,8.0,0.02,0.36,14.0,64.0,15.0,0.0,2.0,64.0,NaN
200005,1263,NaN,11.0,0.06,0.71,24.0,68.0,10.0,0.0,3.0,68.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
202331,0,36.0,7.0,0.00,0.00,72.0,60.0,NaN,53.0,19.0,49.0,0.0
202332,2,36.0,9.0,0.13,0.00,70.0,52.0,NaN,44.0,19.0,48.0,0.0
202333,3,42.0,9.0,0.35,0.00,66.0,52.0,NaN,35.0,20.0,50.0,0.0


In [247]:
epiweek_weather_aqi_syn_cases_df = df

In [248]:
df.to_csv('dataset by week .csv')

In [249]:
df

,cases,Admitted,av_daily_wind_max,precip,av_daily_snow,av_daily_temp,aqi,CO,Ozone,PM10,PM25,NO2
epiweek,,,,,,,,,,,,
200001,97,NaN,12.0,0.05,0.32,31.0,51.0,7.0,0.0,1.0,51.0,NaN
200002,1719,NaN,11.0,0.06,0.22,29.0,54.0,7.0,0.0,1.0,54.0,NaN
200003,632,NaN,10.0,0.05,0.59,18.0,48.0,9.0,0.0,2.0,48.0,NaN
200004,1245,NaN,8.0,0.02,0.36,14.0,64.0,15.0,0.0,2.0,64.0,NaN
200005,1263,NaN,11.0,0.06,0.71,24.0,68.0,10.0,0.0,3.0,68.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
202331,0,36.0,7.0,0.00,0.00,72.0,60.0,NaN,53.0,19.0,49.0,0.0
202332,2,36.0,9.0,0.13,0.00,70.0,52.0,NaN,44.0,19.0,48.0,0.0
202333,3,42.0,9.0,0.35,0.00,66.0,52.0,NaN,35.0,20.0,50.0,0.0


In [250]:
i_df = pd.read_csv('weekly google .csv')

In [251]:
i_df.drop(columns=['Unnamed: 0'], inplace=True)

In [252]:
i_df['date'] = pd.to_datetime(i_df['date'])

In [253]:
i_df['date'] = i_df['date'].apply(lambda x: Week.fromdate(x))

In [254]:
i_df

,date,flu,cough,cold
0,200401,880.264644,372.022059,3881.784871
1,200402,709.409348,331.229860,3790.629287
2,200403,685.939707,289.228760,3744.224168
3,200404,1217.871215,302.004747,3659.188946
4,200405,1140.883644,299.740848,3096.137232
...,...,...,...,...
1029,202339,1277.860262,1191.822409,3689.433580
1030,202340,1339.602284,1228.294742,3710.771255
1031,202341,1242.949688,1248.016096,3724.331102
1032,202342,1254.850155,1288.685504,3907.013168


In [257]:
df = df.join(i_df.rename(columns={'date':'epiweek'}).set_index('epiweek'), how='inner')

In [261]:
df.to_csv('dataset by week .csv')

In [262]:
df

,cases,Admitted,av_daily_wind_max,precip,av_daily_snow,av_daily_temp,aqi,CO,Ozone,PM10,PM25,NO2,flu,cough,cold
epiweek,,,,,,,,,,,,,,,
200401,599,NaN,11.0,0.11,1.80,17.0,41.0,6.0,0.0,3.0,35.0,28.0,880.264644,372.022059,3881.784871
200402,640,NaN,12.0,0.07,0.72,23.0,56.0,5.0,0.0,3.0,52.0,25.0,709.409348,331.229860,3790.629287
200403,399,NaN,11.0,0.04,0.50,15.0,41.0,6.0,0.0,1.0,31.0,31.0,685.939707,289.228760,3744.224168
200404,622,NaN,12.0,0.10,1.23,13.0,37.0,6.0,0.0,1.0,27.0,33.0,1217.871215,302.004747,3659.188946
200405,552,NaN,11.0,0.07,0.69,21.0,62.0,10.0,0.0,2.0,59.0,40.0,1140.883644,299.740848,3096.137232
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202331,0,36.0,7.0,0.00,0.00,72.0,60.0,NaN,53.0,19.0,49.0,0.0,1376.484118,905.153916,3392.215115
202332,2,36.0,9.0,0.13,0.00,70.0,52.0,NaN,44.0,19.0,48.0,0.0,1129.484989,911.538378,3304.090764
202333,3,42.0,9.0,0.35,0.00,66.0,52.0,NaN,35.0,20.0,50.0,0.0,857.542196,930.569414,3289.332925


In [265]:
df[['flu','cough','cold']] = df[['flu','cough','cold']].apply(lambda x: round(x))

In [267]:
df.to_csv('dataset by week .csv')